# *Lab 4 - Netflix Recommendation System*

In [1]:
# Extrating features from Wikipedia

In [2]:
import pandas as pd
import numpy as np

In [11]:
link = 'https://en.wikipedia.org/wiki/List_of_American_films_of_2021'

In [12]:
df1 = pd.read_html(link)[2]
df2 = pd.read_html(link)[3]
df3 = pd.read_html(link)[4]
df4 = pd.read_html(link)[5]

In [13]:
df = pd.concat([df1,df2,df3,df4]).reset_index(drop=True)

In [14]:
import tmdbv3api
import json
import requests

tmdb = tmdbv3api.TMDb()

In [15]:
apiauth = '22396953bf97d94aad77322a4980e2d4'
tmdb.api_key = apiauth

In [16]:
from tmdbv3api import Movie

def get_genre(x):
    genres = []
    result = Movie().search(x)
    movie_id = result[0].id
    response = requests.get(f'https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb.api_key}')
    data_json = response.json()
    if data_json['genres']:
        for i in range(0, len(data_json['genres'])):
            genres.append(data_json['genres'][i]['name'])
        return ' '.join(genres)
    else:
        np.nan

In [17]:
# Extract genres

In [ ]:
df['genres'] = df['Title'].apply(lambda x:get_genre(str(x)))

In [ ]:
df2019 = df[['Title','Cast and crew','genres']].copy()

In [ ]:
# Extract Director

In [ ]:
def get_directors(x):
    if '(director)' in x:
        return x.split('(director); ')[0]
    elif '(director/screenplay)' in x:
        return x.split('(director/screenplay); ')[0]
    else:
        return x.split('(directors); ')[0]

In [ ]:
df2019['directors'] = df2019['Cast and crew'].apply(lambda x:get_directors(str(x)))

In [ ]:
# Extract Actor 1

In [ ]:
def get_actor1(x):
    if 'screenplay)' in x:
        return x.split('screenplay); ')[-1].split(',')[0]

In [ ]:
df2019['actor_1_name'] = df2019['Cast and crew'].apply(lambda x:get_actor1(str(x)))

In [ ]:
# Extract Actor 2

In [ ]:
def get_actor2(x):
    if len(x.split('screenplay);')[-1].split(', '))< 2:
        return np.nan
    else:
        return x.split('screenplay);')[-1].split(',')[1]

In [ ]:
df2019['Cast and crew']

In [ ]:
df2019['actor_2_name'] = df2019['Cast and crew'].apply(lambda x:get_actor2(str(x)))

In [ ]:
df2019['actor_2_name']

In [ ]:
# Extract Actor 3

In [ ]:
def get_actor3(x):
    if len(x.split('screenplay);')[-1].split(', '))< 3:
        return np.nan
    else:
        return x.split('screenplay)')[-1].split(',')[2]

In [ ]:
df2019['actor_3_name'] = df2019['Cast and crew'].apply(lambda x:get_actor3(str(x)))

In [ ]:
df2019.rename(columns={'Title':'movie_title'}, inplace=True)

In [ ]:
df2019_ = df2019.drop('Cast and crew', axis=1).copy()

In [ ]:
df2019_['actor_1_name'].replace(np.nan, 'unknown',inplace=True)

In [ ]:
df2019_['actor_2_name'].replace(np.nan, 'unknown',inplace=True)

In [ ]:
df2019_['actor_3_name'].replace(np.nan, 'unknown',inplace=True)

In [ ]:
df2019_['movie_title'] = df2019_['movie_title'].str.lower()

In [ ]:
df2019_.rename(columns={'directors':'director_name'},inplace=True)

In [ ]:
df2019_['Combined'] = df2019_['movie_title'] + df2019_['actor_1_name']+ df2019_['actor_2_name']+ df2019_['actor_3_name']+df2019_['director_name']+ df2019_['genres']

In [ ]:
df2019_.isna().sum()

In [ ]:
df2019_.to_csv('Processed/FinalData3.csv', index=False)